# Complete Banking Reconciliation Setup - Phase 1
## MinIO S3-backed Iceberg Catalog

## 🎯 Learning Objectives

This notebook demonstrates how to set up Apache Iceberg with a **MinIO S3-compatible object storage catalog** for a banking reconciliation system. You will learn:

### **Apache Iceberg Fundamentals**
- **Catalog System**: Understanding how Iceberg catalogs work with S3/MinIO
- **MinIO Storage**: How Iceberg stores metadata and data in MinIO buckets
- **Table Structure**: The file organization created by Iceberg tables in S3
- **Partitioning**: How Iceberg handles partitioned tables

### **What You'll Build**
- A complete banking reconciliation system with three core tables
- S3-based storage with MinIO integration
- Partitioned tables for efficient querying
- Comprehensive audit trail and validation

### **Key Concepts Explained**

#### **1. Iceberg Catalog with MinIO (S3)**
```python
spark.sql.catalog.minio.type = "hadoop"
spark.sql.catalog.minio.warehouse = "s3a://warehouse/"
```
- **Catalog**: A namespace for tables, schemas, and functions
- **MinIO**: S3-compatible object storage for all table data and metadata
- **Warehouse**: Data and metadata stored in MinIO buckets

#### **2. File Structure Created by Iceberg in S3**

```
s3a://warehouse/banking/table_name/
├── metadata/ # Table metadata and version history
│ ├── v1.metadata.json # Current table schema and properties
│ ├── version-hint.text # Points to latest metadata version
│ └── ..crc # Checksums for data integrity
├── data/ # Actual data files (Parquet format)
│ └── partition_paths/ # Partitioned data organized by partition keys
└── snapshots/ # Table snapshots for time travel
```


#### **3. Partitioning Strategy**
- **source_transactions**: Partitioned by `days(transaction_date)` and `source_system`
- **reconciliation_results**: Partitioned by `days(reconciliation_timestamp)` and `match_status`
- **reconciliation_batches**: No partitioning (small lookup table)

## Phase 1: Foundation Setup


## Step 1: Import Required Libraries

**Purpose**: Import all necessary libraries for Spark, MinIO, and file operations.

**Key Libraries**:
- `pyspark.sql.SparkSession`: Core Spark functionality
- `boto3`: AWS S3/MinIO client for object storage
- `os`, `json`, `datetime`: File and data manipulation utilities

In [1]:
# Import required libraries
!pip install --root-user-action=ignore rich boto3 --quiet
from rich import print
from pyspark.sql import SparkSession
import boto3
from botocore.client import Config
import time
import os
import json
from datetime import datetime

print("✓ All required libraries imported successfully")

✓ All required libraries imported successfully

## Step 2: Stop any existing Spark session

**Purpose**: Ensure a clean Spark environment by stopping any existing sessions.

**Why This Matters**:
- Prevents configuration conflicts
- Ensures fresh Iceberg catalog initialization
- Clears any cached metadata or connections

In [2]:
# Stop any existing Spark session
try:
    SparkSession.builder.getOrCreate().stop()
    print("✓ Stopped existing Spark session")
except:
    print("ℹ No existing Spark session to stop")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/21 13:48:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


✓ Stopped existing Spark session

## Step 3: Create Spark Session with Iceberg + MinIO (S3) Configuration

**Purpose**: Initialize Spark with Apache Iceberg extensions and MinIO S3-compatible catalog configuration.

### **Configuration Breakdown**:

#### **Iceberg Extensions**
```python
spark.sql.extensions = "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions"
```
- Enables Iceberg-specific SQL commands (CREATE TABLE, MERGE, etc.)
- Provides time travel, schema evolution, and partition evolution capabilities

#### **Catalog Configuration**
```python
spark.sql.catalog.minio = "org.apache.iceberg.spark.SparkCatalog"
spark.sql.catalog.minio.type = "hadoop"
spark.sql.catalog.minio.warehouse = "s3a://warehouse/"
spark.hadoop.fs.s3a.endpoint = "http://minio:9000"
spark.hadoop.fs.s3a.access.key = "minio"
spark.hadoop.fs.s3a.secret.key = "minio123"
spark.hadoop.fs.s3a.path.style.access = "true"
spark.hadoop.fs.s3a.impl = "org.apache.hadoop.fs.s3a.S3AFileSystem"
```
- **minio**: Our custom catalog name for S3/MinIO
- **warehouse**: S3 bucket path for table data and metadata

#### **Default Catalog**
- You will use `minio` as the catalog name in all SQL operations.

In [17]:
# Create warehouse directory if it doesn't exist
warehouse_dir = "s3a://warehouse"
print(f"✓ Warehouse directory: {warehouse_dir}")

# Create a new Spark session with the correct configuration
spark = SparkSession.builder \
    .master("spark://spark-master:7077") \
    .appName("Iceberg on MinIO") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.minio", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.minio.type", "hadoop") \
    .config("spark.sql.catalog.minio.warehouse", "s3a://warehouse/") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minio") \
    .config("spark.hadoop.fs.s3a.secret.key", "minio123") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()

print("✓ Spark session created successfully")
print(f"Spark version: {spark.version}")
print(f"Default catalog: {spark.conf.get('spark.sql.defaultCatalog')}")
print(f"Warehouse location: {spark.conf.get('spark.sql.catalog.minio.warehouse')}")

✓ Warehouse directory: s3a://warehouse

✓ Spark session created successfully

Spark version: 3.5.6

Default catalog: local

Warehouse location: s3a://warehouse/

## Step 4: Initialize MinIO Client and Check Health

**Purpose**: Set up MinIO (S3-compatible object storage) for future data operations.

### **MinIO Configuration**:
- **Endpoint**: `http://minio:9000` (Docker service name)
- **Credentials**: `minio`/`minio123` (default development credentials)
- **Region**: `us-east-1` (standard region)
- **Signature Version**: `s3v4` (AWS Signature Version 4)

### **Health Check Strategy**:
- Retry logic with exponential backoff
- Graceful handling of connection failures
- Detailed logging for troubleshooting

In [18]:
# Initialize MinIO client
s3_client = boto3.client(
    's3',
    endpoint_url='http://minio:9000',
    aws_access_key_id='minio',
    aws_secret_access_key='minio123',
    config=Config(signature_version='s3v4'),
    region_name='us-east-1'
)

print("✓ MinIO client initialized")
print(f"Endpoint: http://minio:9000")
print(f"Access Key: minio")
print(f"Signature Version: s3v4")

✓ MinIO client initialized

Endpoint: http://minio:9000

Access Key: minio

Signature Version: s3v4

In [19]:
# Check MinIO health with retries
print("Checking MinIO health...")
max_attempts = 30
attempt = 1

while attempt <= max_attempts:
    try:
        # Try to list buckets to check connectivity
        buckets = s3_client.list_buckets()
        print(f"✓ MinIO is ready! Found {len(buckets['Buckets'])} existing buckets")
        break
    except Exception as e:
        print(f"Waiting for MinIO... (attempt {attempt}/{max_attempts}): {str(e)}")
        if attempt >= max_attempts:
            print("⚠ MinIO failed to start within the expected time. Continuing anyway...")
            break
        time.sleep(2)
        attempt += 1

Checking MinIO health...

✓ MinIO is ready! Found 5 existing buckets

## Step 5: Initialize MinIO Buckets

**Purpose**: Create the required S3 buckets for different stages of data processing.

### **Bucket Strategy**:

| Bucket | Purpose | Data Type |
|--------|---------|-----------|
| `warehouse` | Iceberg table data and metadata | Structured data |
| `raw-data` | Original source files | CSV, JSON, etc. |
| `stage-data` | Intermediate processing data | Transformed data |
| `reconciled-data` | Final reconciliation results | Processed results |

### **Benefits of This Structure**:
- **Data Lineage**: Clear separation of data stages
- **Cost Optimization**: Different retention policies per bucket
- **Security**: Granular access controls per bucket
- **Performance**: Optimized storage for each data type

In [20]:
# List existing buckets
try:
    existing_buckets = [bucket['Name'] for bucket in s3_client.list_buckets()['Buckets']]
    print(f"Existing buckets: {existing_buckets}")
except Exception as e:
    print(f"Error listing buckets: {str(e)}")
    existing_buckets = []

# Define required buckets with their purposes
bucket_purposes = {
    'warehouse': 'Iceberg table data and metadata',
    'raw-data': 'Original source files (CSV, JSON)',
    'stage-data': 'Intermediate processing data',
    'reconciled-data': 'Final reconciliation results'
}

# Create buckets if they don't exist
created_buckets = []
for bucket, purpose in bucket_purposes.items():
    if bucket not in existing_buckets:
        try:
            s3_client.create_bucket(Bucket=bucket)
            created_buckets.append(bucket)
            print(f"✓ Created bucket: {bucket} ({purpose})")
        except Exception as e:
            print(f"✗ Error creating bucket {bucket}: {str(e)}")
    else:
        print(f"ℹ Bucket already exists: {bucket} ({purpose})")

print(f"\n📊 Bucket Summary:")
print(f"- Total buckets: {len(existing_buckets) + len(created_buckets)}")
print(f"- Newly created: {len(created_buckets)}")
print(f"- Already existed: {len(existing_buckets)}")

Existing buckets: ['irisa-ot-warehouse', 'raw-data', 'reconciled-data', 'stage-data', 'warehouse']

ℹ Bucket already exists: warehouse (Iceberg table data and metadata)

ℹ Bucket already exists: raw-data (Original source files (CSV, JSON))

ℹ Bucket already exists: stage-data (Intermediate processing data)

ℹ Bucket already exists: reconciled-data (Final reconciliation results)

📊 Bucket Summary:

- Total buckets: 5

- Newly created: 0

- Already existed: 5

## Step 6: Create Banking Namespace

**Purpose**: Create a logical namespace to organize related tables in MinIO.

### **Namespace Benefits**:
- **Organization**: Groups related tables together
- **Access Control**: Apply permissions at namespace level
- **Naming**: Avoid table name conflicts
- **Discovery**: Easier to find related tables

### **S3 Impact**:
The namespace creates a directory structure in your MinIO bucket:
```
s3a://warehouse/banking/
├── source_transactions/
├── reconciliation_results/
└── reconciliation_batches/
```


In [21]:
# Create the banking namespace
try:
    spark.sql("CREATE NAMESPACE IF NOT EXISTS minio.banking")
    print("✓ Created namespace: minio.banking")
    print(f"Namespace location: {warehouse_dir}/banking")
except Exception as e:
    print(f"✗ Error creating namespace: {str(e)}")

✓ Created namespace: minio.banking

Namespace location: s3a://warehouse/banking

## Step 6b: Verify Namespace Creation

**Purpose**: Confirm that the namespace directory structure exists in your MinIO bucket.

> Note: You can use MinIO Console (http://localhost:9001) or `mc` CLI to browse the S3 structure:
> - `s3a://warehouse/banking/`

## Step 7: Create Iceberg Tables in MinIO

**Purpose**: Create the core tables for the banking reconciliation system in the MinIO S3 bucket.

### **Table Design Strategy**:

#### **1. source_transactions**
- **Purpose**: Store all transaction data from different source systems
- **Partitioning**: By `days(transaction_date)` and `source_system`
- **Benefits**: Efficient querying by date range and source
- **Schema**: Comprehensive transaction metadata

#### **2. reconciliation_results**
- **Purpose**: Store reconciliation outcomes and discrepancies
- **Partitioning**: By `days(reconciliation_timestamp)` and `match_status`
- **Benefits**: Easy analysis of reconciliation performance
- **Schema**: Detailed discrepancy tracking

#### **3. reconciliation_batches**
- **Purpose**: Track reconciliation batch metadata
- **Partitioning**: None (small lookup table)
- **Benefits**: Batch-level monitoring and reporting
- **Schema**: Batch execution metadata

> All tables will be created under the `minio.banking` namespace and stored in your MinIO S3 bucket.

In [22]:
# Create source_transactions table
print("Creating source_transactions table...")
try:
    spark.sql("""
    CREATE TABLE IF NOT EXISTS minio.banking.source_transactions (
      transaction_id STRING,
      source_system STRING,
      transaction_date TIMESTAMP,
      amount DECIMAL(18,2),
      account_id STRING,
      transaction_type STRING,
      reference_id STRING,
      status STRING,
      payload STRING,
      created_at TIMESTAMP,
      processing_timestamp TIMESTAMP
    )
    USING iceberg
    PARTITIONED BY (days(transaction_date), source_system)
    """)
    print("✓ Created table: minio.banking.source_transactions")
    print("  - Partitioned by: days(transaction_date), source_system")
    print("  - Purpose: Store all transaction data from different sources")
except Exception as e:
    print(f"✗ Error creating source_transactions table: {str(e)}")

Creating source_transactions table...

✓ Created table: minio.banking.source_transactions

- Partitioned by: days(transaction_date), source_system

- Purpose: Store all transaction data from different sources

In [24]:
# Create reconciliation_results table
print("Creating reconciliation_results table...")
try:
    spark.sql("""
    CREATE TABLE IF NOT EXISTS minio.banking.reconciliation_results (
      reconciliation_id STRING,
      batch_id STRING,
      primary_transaction_id STRING,
      secondary_transaction_id STRING,
      match_status STRING,
      discrepancy_type STRING,
      discrepancy_amount DECIMAL(18,2),
      reconciliation_timestamp TIMESTAMP,
      notes STRING
    )
    USING iceberg
    PARTITIONED BY (days(reconciliation_timestamp), match_status)
    """)
    print("✓ Created table: minio.banking.reconciliation_results")
    print("  - Partitioned by: days(reconciliation_timestamp), match_status")
    print("  - Purpose: Store reconciliation outcomes and discrepancies")
except Exception as e:
    print(f"✗ Error creating reconciliation_results table: {str(e)}")

Creating reconciliation_results table...

✓ Created table: minio.banking.reconciliation_results

- Partitioned by: days(reconciliation_timestamp), match_status

- Purpose: Store reconciliation outcomes and discrepancies

In [23]:
# Create reconciliation_batches table
print("Creating reconciliation_batches table...")
try:
    spark.sql("""
    CREATE TABLE IF NOT EXISTS minio.banking.reconciliation_batches (
      batch_id STRING,
      reconciliation_date TIMESTAMP,
      source_systems ARRAY<STRING>,
      start_date TIMESTAMP,
      end_date TIMESTAMP,
      status STRING,
      total_transactions BIGINT,
      matched_count BIGINT,
      unmatched_count BIGINT,
      created_at TIMESTAMP,
      completed_at TIMESTAMP
    )
    USING iceberg
    """)
    print("✓ Created table: minio.banking.reconciliation_batches")
    print("  - Partitioned by: None (small lookup table)")
    print("  - Purpose: Track reconciliation batch metadata")
except Exception as e:
    print(f"✗ Error creating reconciliation_batches table: {str(e)}")

Creating reconciliation_batches table...

✓ Created table: minio.banking.reconciliation_batches

- Partitioned by: None (small lookup table)

- Purpose: Track reconciliation batch metadata

## Step 8: Verify Tables and Audit Setup

**Purpose**: Validate that all tables were created correctly in MinIO and examine the Iceberg file structure.

### **What We'll Verify**:
1. **Table Existence**: Confirm all tables are created in the `minio.banking` namespace
2. **File Structure**: Examine Iceberg metadata and data directories in S3
3. **Schema Validation**: Check table schemas are correct
4. **Accessibility**: Test basic queries on each table

In [25]:
# List tables to verify
print("📋 Verifying created tables...")
tables_df = spark.sql("SHOW TABLES IN minio.banking")
tables_df.show()

# Count tables
table_count = tables_df.count()
print(f"\n📊 Table Summary:")
print(f"- Total tables in banking namespace: {table_count}")
print(f"- Expected tables: 3")
print(f"- Status: {'✓ PASS' if table_count >= 3 else '✗ FAIL'}")

📋 Verifying created tables...

+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
|  banking|reconciliation_ba...|      false|
|  banking|reconciliation_re...|      false|
|  banking| source_transactions|      false|
+---------+--------------------+-----------+



📊 Table Summary:

- Total tables in banking namespace: 3

- Expected tables: 3

- Status: ✓ PASS

In [12]:
# Test a simple query on each table
print("🔍 Testing table accessibility...")

tables_to_test = [
    'minio.banking.source_transactions',
    'minio.banking.reconciliation_results',
    'minio.banking.reconciliation_batches'
]

for table in tables_to_test:
    try:
        result = spark.sql(f"SELECT COUNT(*) as count FROM {table}")
        count = result.collect()[0]['count']
        print(f"✓ {table}: {count} rows")
    except Exception as e:
        print(f"✗ {table}: Error - {str(e)}")

🔍 Testing table accessibility...

✓ minio.banking.source_transactions: 0 rows

✓ minio.banking.reconciliation_results: 0 rows

✓ minio.banking.reconciliation_batches: 0 rows

## Step 9: Examine Iceberg File Structure in MinIO

**Purpose**: Understand how Iceberg organizes files and metadata in your MinIO S3 bucket.

### **Iceberg File Organization in S3**:

Each Iceberg table creates this structure in your MinIO bucket:
```
s3a://warehouse/banking/table_name/
├── metadata/              # Table metadata and version history
│   ├── v1.metadata.json  # Current table schema and properties
│   ├── version-hint.text # Points to latest metadata version
│   └── .*.crc           # Checksums for data integrity
├── data/                # Actual data files (Parquet format)
│   └── partition_paths/ # Partitioned data organized by partition keys
└── snapshots/           # Table snapshots for time travel
```
> Use the MinIO Console (http://localhost:9001) or mc CLI to browse and inspect these files.

### **Key Files Explained**:
- **v1.metadata.json**: Contains table schema, partitioning, and properties
- **version-hint.text**: Points to the current metadata version
- **.crc files**: Checksums to ensure data integrity
- **data/**: Contains actual Parquet files with the data
- **snapshots/**: Enables time travel and rollback capabilities

In [26]:
bucket_name = 'warehouse'
namespace_prefix = 'banking/'

def list_s3_prefix(prefix, title):
    print(f"\n🔍 {title}")
    response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix, Delimiter='/')
    if 'Contents' in response:
        for obj in response['Contents']:
            print(obj['Key'])
    else:
        print("No objects found.")

print("📁 Examining Iceberg file structure...")

# Banking namespace structure
list_s3_prefix(namespace_prefix, "Banking namespace structure")

# source_transactions table structure
list_s3_prefix(namespace_prefix + 'source_transactions/', "source_transactions table structure")

# source_transactions metadata
list_s3_prefix(namespace_prefix + 'source_transactions/metadata/', "source_transactions metadata")

# reconciliation_results table structure
list_s3_prefix(namespace_prefix + 'reconciliation_results/', "reconciliation_results table structure")

# reconciliation_batches table structure
list_s3_prefix(namespace_prefix + 'reconciliation_batches/', "reconciliation_batches table structure")

📁 Examining Iceberg file structure...

🔍 Banking namespace structure

No objects found.

🔍 source_transactions table structure

No objects found.

...

banking/source_transactions/metadata/version-hint.text

🔍 reconciliation_results table structure

No objects found.

🔍 reconciliation_batches table structure

No objects found.

In [14]:
# Examine the metadata file to understand table schema and properties
bucket_name = 'warehouse'
metadata_prefix = 'banking/source_transactions/metadata/'

def read_s3_object(key, title):
    print(f"\n🔍 {title}")
    try:
        obj = s3_client.get_object(Bucket=bucket_name, Key=key)
        content = obj['Body'].read().decode('utf-8')
        print(content)
    except Exception as e:
        print(f"Error reading {key}: {e}")

print("📄 Examining table metadata...")

# Read v1.metadata.json
read_s3_object(metadata_prefix + 'v1.metadata.json', "source_transactions metadata content")

# Read version-hint.text
read_s3_object(metadata_prefix + 'version-hint.text', "version hint file")

📄 Examining table metadata...

🔍 source_transactions metadata content

{"format-version":2,"table-uuid":"c840d44b-9064-460e-a80a-30f585faf094","location":"s3a://warehouse/banking/source_
transactions","last-sequence-number":0,"last-updated-ms":1753105692669,"last-column-id":11,"current-schema-id":0,"s
chemas":[{"type":"struct","schema-id":0,"fields":[{"id":1,"name":"transaction_id","required":false,"type":"string"}
,{"id":2,"name":"source_system","required":false,"type":"string"},{"id":3,"name":"transaction_date","required":fals
e,"type":"timestamptz"},{"id":4,"name":"amount","required":false,"type":"decimal(18, 
2)"},{"id":5,"name":"account_id","required":false,"type":"string"},{"id":6,"name":"transaction_type","required":fal
se,"type":"string"},{"id":7,"name":"reference_id","required":false,"type":"string"},{"id":8,"name":"status","requir
ed":false,"type":"string"},{"id":9,"name":"payload","required":false,"type":"string"},{"id":10,"name":"created_at",
"required":false,"type":"timestamptz"},{"id":11,"name":"processing_timestamp","required":false,"type":"timestamptz"
}]}],"default-spec-id":0,"partition-specs":[{"spec-id":0,"fields":[{"name":"transaction_date_day","transform":"day"
,"source-id":3,"field-id":1000},{"name":"source_system","transform":"identity","source-id":2,"field-id":1001}]}],"l
ast-partition-id":1001,"default-sort-order-id":0,"sort-orders":[{"order-id":0,"fields":[]}],"properties":{"owner":"
spark","write.parquet.compression-codec":"zstd"},"current-snapshot-id":-1,"refs":{},"snapshots":[],"statistics":[],
"partition-statistics":[],"snapshot-log":[],"metadata-log":[]}

🔍 version hint file

1

## Step 10: Phase 1 Summary and Next Steps

**Purpose**: Provide a comprehensive summary of what was accomplished and prepare for the next phase.

### **What We've Accomplished**:

#### **✅ Infrastructure Setup**
- Spark session with Iceberg extensions
- Local catalog with Hive metastore
- MinIO object storage buckets

#### **✅ Data Architecture**
- Banking namespace for organization
- Three core tables with proper schemas
- Partitioning strategy for performance

#### **✅ File Structure**
- Iceberg metadata and data directories
- Proper table organization
- Version control and checksums

### **Next Phase Preview**:
- **Data Generation**: Create sample transaction data
- **Data Ingestion**: Load data into Iceberg tables
- **Reconciliation Logic**: Implement matching algorithms
- **Testing**: Validate the complete system

In [28]:
# Generate setup summary
print("\n📚 Learning Summary:")
print("✅ Understanding of Iceberg Hadoop catalog with Hive Style Partitioning ")
print("✅ Knowledge of Iceberg file structure and metadata organization")
print("✅ Experience with table partitioning and schema design")
print("✅ Familiarity with MinIO object storage integration")
print("\n🚀 Next: Run Phase 2 notebook for data generation and population.")

📚 Learning Summary:

✅ Understanding of Iceberg Hadoop catalog with Hive Style Partitioning

✅ Knowledge of Iceberg file structure and metadata organization

✅ Experience with table partitioning and schema design

✅ Familiarity with MinIO object storage integration

🚀 Next: Run Phase 2 notebook for data generation and population.

In [27]:
spark.stop()